# Random Forests


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
)
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
from binning import bin

Read the train and test datasets


In [2]:
train_df = pd.read_csv("Training_set.csv")
test_df = pd.read_csv("Testing_set.csv")

Convert training set OHE columns to int8


In [3]:
columns_to_convert = train_df.columns[
    17:-1
]  # Columns from 17th column up to but excluding the last column
train_df[columns_to_convert] = train_df[columns_to_convert].astype("int8")
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385516 entries, 0 to 385515
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: float64(12), int64(6), int8(4055)
memory usage: 1.5 GB


Convert testing set OHE columns to int8


In [4]:
columns_to_convert = test_df.columns[
    17:-1
]  # Columns from 17th column up to but excluding the last column
test_df[columns_to_convert] = test_df[columns_to_convert].astype("int8")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96379 entries, 0 to 96378
Columns: 4073 entries, ScheduledArrTime to DepDelay
dtypes: float64(12), int64(6), int8(4055)
memory usage: 385.9 MB


Get the training features and label


In [5]:
X_train = train_df.iloc[:, :-1].to_numpy()
y_train = train_df.iloc[:, -1].to_numpy()

Get the testing features and label


In [6]:
X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

Initialize a 5-fold cross-validation object


In [12]:
kf = KFold(n_splits=5)

Bin the label


In [7]:
y_train_binned = bin(y_train)
y_test_binned = bin(y_test)

Create a new RandomForest classifier with the best parameter values


In [35]:
rfc = RandomForestClassifier(n_jobs=5, class_weight={0: 10, 1: 1}, max_depth=30)

Initialize scores lists


In [36]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

Train the model using 5-fold cross-validation


In [37]:
fold = 1
for train, test in kf.split(X_train, y_train_binned):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    rfc.fit(X_train[train], y_train_binned[train])

    # Predict on the test set
    predictions = rfc.predict(X_train[test])

    # Evaluate the model
    precision = precision_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train_binned[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train_binned[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
Precision = 0.6336302750812993
Recall = 0.6519635816559453
Accuracy = 0.6519635816559453
F1 score = 0.6307974329487073

##### FOLD: 2 #####
Precision = 0.6319363566999248
Recall = 0.6504286473937461
Accuracy = 0.6504286473937461
F1 score = 0.6302243890923406

##### FOLD: 3 #####
Precision = 0.6305509769223417
Recall = 0.6490149540225413
Accuracy = 0.6490149540225413
F1 score = 0.6287699511751491

##### FOLD: 4 #####
Precision = 0.633670652200433
Recall = 0.6524778542987951
Accuracy = 0.6524778542987951
F1 score = 0.6316458682703031

##### FOLD: 5 #####
Precision = 0.632663987674465
Recall = 0.6508047676484702
Accuracy = 0.6508047676484702
F1 score = 0.6309295046142636



Print the mean scores of the folds


In [38]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.6324904497156927
Mean Recall = 0.6509379610038996
Mean Accuracy = 0.6509379610038996
Mean F1 score = 0.6304734292201527


Make predictions on the test set


In [39]:
predictions = rfc.predict(X_test)

Evaluate the model


In [40]:
print(classification_report(y_true=y_test_binned, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.54      0.36      0.43     35520
           1       0.69      0.82      0.75     60859

    accuracy                           0.65     96379
   macro avg       0.61      0.59      0.59     96379
weighted avg       0.63      0.65      0.63     96379

